In [1]:
import os
from langchain_community.embeddings import GPT4AllEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = GPT4AllEmbeddings()
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [2]:
import os
import json
import numpy as np
import torch
from torch_geometric.data import Data
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import openai
import os
from dotenv import load_dotenv

load_dotenv()


c:\Users\Nhat Minh\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


True

In [3]:
from openai import OpenAI

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key = "",
)

def ask_gpt(messages: list[dict], model="gpt-4o") -> str:
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=512
    )
    return response.choices[0].message.content

# 2) System prompts
sys_res = (
    "Xếp hạng lại danh sách restaurants dựa trên sở thích của người dùng. "
    "Chỉ trả về danh sách tên các restaurant, cách nhau bằng dấu phẩy."
)

sys_explain = (
    "Bạn là một chuyên gia về phân tích. "
    "Dựa vào những sở thích của người dùng và thứ tự bạn đã xếp hạng, "
    "hãy giải thích ngắn gọn lý do tại sao bạn có kết quả đó."
)


sys_res = "Xếp hạng lại danh sách restaurants dựa trên sở thích của người dùng. Chỉ trả về danh sách tên các restaurant, cách nhau bằng dấu phẩy."

sys_key = "Chỉ trả về danh sách tên các restaurant, cách nhau bằng dấu phẩy."

In [5]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

c:\Users\Nhat Minh\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
with open('edinburgh-keywords_train.json', 'r', encoding="utf-8") as f:
    train_data = json.load(f)

keywords = list(train_data['np2count'].keys())
keyword_set = set(keywords)

def extract_users(info):
    l_user, user2kw = [], []
    for ii in info:
        lus = info[ii]
        for u in lus:
            if u not in l_user:
                l_user.append(u)
                user2kw.append([])
            idx = l_user.index(u)
            user2kw[idx].append(ii)
    return l_user, user2kw


In [7]:
train_users, train_users2kw = extract_users(train_data['np2users'])

In [8]:
restaurant_set = set()
listres = []
for kw in train_data['np2rests'].keys():
    listres.extend(train_data['np2rests'][kw].keys())
restaurant_set = set(listres)

keyword_set = list(keyword_set)
restaurant_set = list(restaurant_set)
restaurants = len(listres)
num_keywords = len(keyword_set)
num_restaurants = len(restaurant_set)
a = np.zeros((num_keywords, num_restaurants))

for kw in train_data['np2rests'].keys():
    for res in train_data['np2rests'][kw].keys():
        idx_kw = keyword_set.index(kw)
        idx_res = restaurant_set.index(res)
        a[idx_kw][idx_res] = 1


In [9]:
keyword_embeddings = model.encode(list(keyword_set))

In [10]:
# Load dữ liệu test
with open('edinburgh-keywords_test.json', 'r', encoding="utf-8") as r:
    test_data = json.load(r)

user_keywords = list(test_data['np2reviews'].keys())
user_keywords_list = list(user_keywords)


In [11]:
test_users, test_users2kw = extract_users(test_data['np2users'])
test_keywords = [kw for sublist in test_users2kw for kw in sublist]
test_keyword_embeddings = model.encode(test_keywords)
similarity_scores = cosine_similarity(test_keyword_embeddings, keyword_embeddings)


In [12]:
filtered_keywords = []
for i, user_kw in enumerate(test_users2kw):
    updated_user_kw = []
    for kw in user_kw:
        if kw not in keyword_set:
            test_idx = test_keywords.index(kw)
            sim_scores = similarity_scores[test_idx]
            best_match_idx = np.argmax(sim_scores)
            best_match_keyword = keyword_set[best_match_idx]
            updated_user_kw.append(best_match_keyword)
        else:
            updated_user_kw.append(kw)
    filtered_keywords.append(updated_user_kw)

In [13]:
test_users2kw = filtered_keywords

In [14]:
results = []
for kw in test_users2kw:
    t = np.zeros((1, len(keyword_set)))
    keywords_subset = kw[:10]
    for key in keywords_subset:
        if key in keyword_set:
            idx_kw = keyword_set.index(key)
            t[0][idx_kw] = 1
    R = np.dot(t, a)
    result = np.argsort(R[0])[::-1][:10]
    results.append(result)

In [25]:
sys_explain = (
    "Bạn là một chuyên gia về phân tích"
    "Hãy dựa vào những sở thích của người dùng và danh sách mà bạn đã xếp hạng"
    "hãy giải thích ngắn gọn về lý do tại sao bạn lại có những xếp hạng như vậy"
)

In [28]:
def GPT_re_rank_with_explanation(user_id, candidate_restaurants, user_keywords):
    prompt_rank = (
        f"Người dùng có sở thích: {', '.join(user_keywords[:5])}.\n"
        f"Candidate restaurants: {', '.join(candidate_restaurants[:5])}.\n"
        "Hãy xếp hạng lại theo mức độ phù hợp, chỉ trả về tên, cách nhau dấu phẩy."
    )
    messages_rank = [
        {"role": "system", "content": sys_res},
        {"role": "user",   "content": prompt_rank},
    ]
    ranked_text = ask_gpt(messages_rank)  
    re_ranked = [r.strip() for r in ranked_text.split(",") if r.strip() in candidate_restaurants]
    if not re_ranked:
        re_ranked = candidate_restaurants

    prompt_explain = (
        f"Nhà hàng đã được xếp hạng như sau: {', '.join(re_ranked)}.\n"
        f"Sở thích người dùng: {', '.join(user_keywords[:5])}.\n"
        "Vui lòng cho biết lý do ngắn gọn tại sao thứ tự này phù hợp nhất."
    )
    messages_explain = [
        {"role": "system", "content": sys_explain},
        {"role": "user",   "content": prompt_explain},
    ]
    explanation = ask_gpt(messages_explain)

    return re_ranked, explanation

final_results = []
final_explanations = []
for idx, (user, candidate_indices) in enumerate(zip(test_users, results)):
    candidate_restaurants = [restaurant_set[i] for i in candidate_indices]
    user_kw = test_users2kw[idx]
    ranked, reason = GPT_re_rank_with_explanation(user, candidate_restaurants, user_kw)
    final_results.append(ranked)
    final_explanations.append(reason)
    print(f"User {user} → Ranking: {ranked}")
    print(f"Explanation: {reason}\n")


User yfXqZkU5iXE07GSHzdsQBA → Ranking: ['E7s0QreUJ2H9r0Ls1yDeiQ', '8PmbCVEjWGPouD1Tst6FsQ', 'QMySP-qHdwQNZY0gDba0ng', 'xbCNlp2vWixy2W1WsfGv8g', 'z-3ZDmVbHZjDrYZE42yKZA']
Explanation: Thứ tự xếp hạng các nhà hàng phù hợp nhất dựa trên sở thích của người dùng theo cách sau:

1. **E7s0QreUJ2H9r0Ls1yDeiQ**: Nhà hàng này có thể có món ăn hấp dẫn và đa dạng, phù hợp với tiêu chí "food" và "menu" được người dùng đánh giá cao. Có thể họ có không gian phù hợp cho việc ăn uống với bạn bè (flatmates).

2. **8PmbCVEjWGPouD1Tst6FsQ**: Nơi này cũng được đánh giá cao về thực đơn (menu) và chất lượng món ăn (food). Nhà hàng có thể cung cấp dịch vụ thoải mái, thích hợp cho nhóm bạn (flatmates), ví dụ như có không gian thoải mái hoặc chế độ phục vụ tốt gần cửa ra vào (door).

3. **QMySP-qHdwQNZY0gDba0ng**: Nhà hàng này có thực đơn (menu) hấp dẫn với chất lượng tốt và có thể thuận tiện cho việc tụ tập bạn bè (flatmates). Có lẽ dịch vụ tại cửa ra vào (door) cũng được chú ý.

4. **xbCNlp2vWixy2W1WsfGv8g**:

In [29]:
# Hàm trả về kết quả dạng dictionary để sử dụng cho hệ thống
def generate_results(test_users, results, test_users2kw, restaurant_set, re_ranked):
    output_data = {}
    for idx, (user, restaurant_indices) in enumerate(zip(test_users, results)):
        user_data = {}
        user_keywords = test_users2kw[idx]
        candidate_restaurants = [restaurant_set[i] for i in restaurant_indices]
        re_ranked_restaurants = re_ranked[idx]
        positions = [str(i) for i in restaurant_indices]
        user_data["kw"] = user_keywords[:5]
        user_data["candidate"] = re_ranked_restaurants[:5]
        user_data["positions"] = positions[:5]
        output_data[user] = user_data
    return output_data 

result_dict = generate_results(test_users, results, test_users2kw, restaurant_set, final_results)
print(result_dict)

{'yfXqZkU5iXE07GSHzdsQBA': {'kw': ['flatmates', 'food', 'while', 'menu', 'door'], 'candidate': ['E7s0QreUJ2H9r0Ls1yDeiQ', '8PmbCVEjWGPouD1Tst6FsQ', 'QMySP-qHdwQNZY0gDba0ng', 'xbCNlp2vWixy2W1WsfGv8g', 'z-3ZDmVbHZjDrYZE42yKZA'], 'positions': ['879', '404', '281', '139', '612']}, 'hutJzKEYHuVq6CP-XSARgg': {'kw': ['flatmates', 'food', 'while', 'menu', 'fried rice'], 'candidate': ['adI5BlJCtvWq2uAwmyndXA', 'aIs7Tt3pw6ymj3FOPPR4Ng', '5Iin0tl6QJT_TyL1BldtyA', 'sxuEde0_sVFY-Uk9gpHy_g', 'hJNjKc4sKgWG__KitlJ1ag'], 'positions': ['133', '339', '811', '157', '516']}, '2UkZKQBZVuroUBKYs9WzeQ': {'kw': ['food', 'menu', 'minutes', 'wait', 'restaurant'], 'candidate': ['I1j1kEbQ5BliKsjzl5etVA', 'J_KI8rJa_dSwTglUbwFcdw', 'Q0fcX_1wvdmffqEPa246rg', 'xWuzWH8qYaa75REqVd9Isw', 'xbCNlp2vWixy2W1WsfGv8g'], 'positions': ['433', '181', '595', '139', '98']}, 'hihNuqYNKDwAwHeGNURE_g': {'kw': ['food', 'spicy sauce', 'meal', 'more time', 'queen'], 'candidate': ['ZMcd7w-yDhFRLSwvJBV1VQ', '6MR9GZjWjLAS3s39YdDXKw', 'Q0fcX

In [ ]:
# Hàm lưu kết quả vào file JSON để kiểm tra nhanh
def save_rerank_results_to_json(test_users, results, test_users2kw, restaurant_set, re_ranked, file_path='./data/Output_GPT(re_rank).json'):
    output_data = {}
    for idx, (user, restaurant_indices) in enumerate(zip(test_users, results)):
        user_data = {}  
        user_keywords = test_users2kw[idx]
        candidate_restaurants = [restaurant_set[i] for i in restaurant_indices]
        re_ranked_restaurants = re_ranked[idx]
        positions = [str(i) for i in restaurant_indices]
        user_data["kw"] = user_keywords[:10]
        user_data["candidate"] = re_ranked_restaurants[:10]
        user_data["positions"] = positions[:10]
        output_data[user] = user_data
    with open(file_path, mode="w", encoding="utf-8") as json_file:
        json.dump(output_data, json_file, ensure_ascii=False, indent=4)
    print(f"Results saved to: {file_path}")

save_rerank_results_to_json(test_users, results, test_users2kw, restaurant_set, final_results)


Results saved to: ./data/Output_GPT(re_rank).json


In [ ]:
def generate_full_results(test_users, results, test_users2kw, restaurant_set, re_ranked, explanations):
    output_data = {}
    for idx, user in enumerate(test_users):
        kws = test_users2kw[idx][:5]
        cand = re_ranked[idx][:5]
        # orig = [restaurant_set[i] for i in results[idx]][:5]
        exp = explanations[idx]
        output_data[user] = {
            "kw": kws,
            "candidate": cand,
            "explanation": exp
        }
    return output_data

generate_full_results(test_users, results, test_users2kw, restaurant_set, final_results, final_explanations)

{'yfXqZkU5iXE07GSHzdsQBA': {'kw': ['flatmates',
   'food',
   'while',
   'menu',
   'door'],
  'candidate': ['E7s0QreUJ2H9r0Ls1yDeiQ',
   '8PmbCVEjWGPouD1Tst6FsQ',
   'QMySP-qHdwQNZY0gDba0ng',
   'xbCNlp2vWixy2W1WsfGv8g',
   'z-3ZDmVbHZjDrYZE42yKZA'],
  'explanation': 'Thứ tự xếp hạng các nhà hàng phù hợp nhất dựa trên sở thích của người dùng theo cách sau:\n\n1. **E7s0QreUJ2H9r0Ls1yDeiQ**: Nhà hàng này có thể có món ăn hấp dẫn và đa dạng, phù hợp với tiêu chí "food" và "menu" được người dùng đánh giá cao. Có thể họ có không gian phù hợp cho việc ăn uống với bạn bè (flatmates).\n\n2. **8PmbCVEjWGPouD1Tst6FsQ**: Nơi này cũng được đánh giá cao về thực đơn (menu) và chất lượng món ăn (food). Nhà hàng có thể cung cấp dịch vụ thoải mái, thích hợp cho nhóm bạn (flatmates), ví dụ như có không gian thoải mái hoặc chế độ phục vụ tốt gần cửa ra vào (door).\n\n3. **QMySP-qHdwQNZY0gDba0ng**: Nhà hàng này có thực đơn (menu) hấp dẫn với chất lượng tốt và có thể thuận tiện cho việc tụ tập bạn bè (fl

In [47]:
def save_full_results_to_json(test_users, results, test_users2kw, restaurant_set, re_ranked, explanations, 
                              file_path='./data/Output_GPT(explanation).json'):
    output_data = generate_full_results(
        test_users, results, test_users2kw, restaurant_set, re_ranked, explanations
    )
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, mode="w", encoding="utf-8") as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)
    print(f"Full results saved to: {file_path}")

save_full_results_to_json(test_users, results, test_users2kw, restaurant_set, final_results, final_explanations)



Full results saved to: ./data/Output_GPT(explanation).json
